In [18]:
import pandas as pd
import requests
import json

# Caminho para o arquivo
file_path = r'G:\Meu Drive\Caetano\Renda\Snov.io\Data Analysis\SalesOps\planilhas\ChurnedEmails.xlsx'

# Leitura do arquivo Excel
df = pd.read_excel(file_path, sheet_name= "semana29.07")

# Exibindo as primeiras linhas do DataFrame
print(df.head())

# Authentication
client_id = 'c9e265e0b019d7239392d3b2bdba8d58'
client_secret = '94b42cd2d9a6503449bce297698d3136'

def get_access_token():
    params = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }

    res = requests.post('https://api.snov.io/v1/oauth/access_token', data=params)
    resText = res.text.encode('ascii', 'ignore')

    return json.loads(resText)['access_token']

# Get profile by email
def get_profile_by_email(email):
    try:
        token = get_access_token()
        params = {'access_token': token,
                  'email': email}

        res = requests.post('https://api.snov.io/v1/get-profile-by-email', data=params)
        res.raise_for_status()  # Levanta um erro para códigos de status HTTP 4xx/5xx
        return json.loads(res.text)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching profile for {email}: {e}")
        return None

# Processando cada email no DataFrame e armazenando os perfis
profiles = []

for index, row in df.iterrows():
    email = row['Email']
    profile = get_profile_by_email(email)
    if profile and profile.get('success'):
        social_links = profile.get('social', [])
        linkedin_link = next((link['link'] for link in social_links if link['type'] == 'linkedIn'), '')
        first_name = profile.get('firstName', '')
        last_name = profile.get('lastName', '')
        current_jobs = profile.get('currentJobs', [])
        position = current_jobs[0]['position'] if current_jobs else ''
    else:
        linkedin_link = ''
        first_name = ''
        last_name = ''
        position = ''

    enriched_data = {
        'Email': email,
        'Full Name': row['Full Name'],
        'Purchase': row['Purchase'],
        'Plan': row['Plan'],
        'Payment': row['Payment'],
        'SocialLink': linkedin_link,
        'Nome': first_name,
        'Sobrenome': last_name,
        'Cargo': position
    }
    profiles.append(enriched_data)
    print(f"Profile for {email}: {linkedin_link}, {first_name}, {last_name}, {position}")

# Convertendo a lista de perfis em um DataFrame
profiles_df = pd.DataFrame(profiles)

# Salvando em um arquivo Excel
output_file_path = r'G:\Meu Drive\Caetano\Renda\Snov.io\Data Analysis\SalesOps\planilhas\EnrichedChurnedEmails.xlsx'
profiles_df.to_excel(output_file_path, index=False)

print(f"Enriched data saved to {output_file_path}")


                            Email        Full Name   Purchase  Plan    Payment
0      fernando.veloso@vidibr.com  Fernando Veloso 2024-02-29  1000  28.00 USD
1  leonardo@kennedyviagens.com.br  Leonardo Bastos 2024-02-29  5000  70.00 USD
2           gabi@dehumanos.com.br   GABRIELA NUNES 2024-02-29  5000  99.00 USD
3        fabricio@corremkt.com.br   Fabrício Gomes 2024-02-29  1000  39.00 USD
4        felipe.paz@evolux.net.br       Felipe Paz 2024-02-29  1000  39.00 USD
Profile for fernando.veloso@vidibr.com: https://www.linkedin.com/in/fernando-veloso1995, Fernando, Veloso, Coordenador Comercial, CRM e Desenvolvimento de Novos Negócios
Profile for leonardo@kennedyviagens.com.br: , , , 
Profile for gabi@dehumanos.com.br: , , , 
Profile for fabricio@corremkt.com.br: https://www.linkedin.com/in/fabriciomgomes-/, Fabrício, Gomes, SDR
Profile for felipe.paz@evolux.net.br: https://www.linkedin.com/in/felipepaz/, Felipe, Paz, Diretor comercial
Profile for accounting@ottomantech.net: , , , 
Pr